<a href="https://colab.research.google.com/github/marcory-hub/swift-yolo/blob/main/Swift_YOLO_192_gdrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train a Swift-YOLO 192×192 model with a dataset from Google Drive
This notebook trains a **Swift-YOLO** (tiny, 192×192) object-detection model using [ModelAssistant](https://github.com/Seeed-Studio/ModelAssistant) (SSCMA). Your dataset lives in Google Drive; the notebook copies it into Colab and runs training.

**Category:** Object Detection

**Algorithm:** [Swift-YOLO](configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py)

**Dataset:** Custom dataset from Google Drive (requires Gmail account)

---

## Setup instructions

Do these steps **before** running the notebook.

### Step 1: Enable GPU

1. In Colab: **Runtime** → **Change runtime type**
2. Set **Hardware accelerator** to **GPU** (choose **A100** if available for faster training)
3. Click **Save**

**Expected environment** (check the first code cell output):
- GPU: T4 or higher (A100 preferred for long runs)
- Python 3.11.x

---

### Step 2: Prepare your dataset in COCO format

1. **Folder structure** — Use this layout (exact names). Each of `train/` and `valid/` must contain images and one `_annotations.coco.json`:

```
dataset
├── train
│   ├── _annotations.coco.json
│   ├── image1.jpg
│   ├── image2.jpg
│   └── ...
└── valid
    ├── _annotations.coco.json
    ├── image3.jpg
    ├── image4.jpg
    └── ...
```

2. **Zip the dataset** — Create `dataset.zip` from the `dataset` folder. On macOS/Linux you can exclude metadata with:

   `zip -r dataset.zip dataset -i '*.jpg' '*.jpeg' '*.png' '*.json'`

3. **Upload to Google Drive** — Put `dataset.zip` in the root directory. Optionally put a custom pretrained checkpoint there too (e.g. `best_coco_bbox_mAP_epoch_100.pth`) if you want to fine-tune from your own model.

---

### Step 3: Set number of classes

⚠️ **Update the class count to match your dataset.**

- **Default in this notebook:** 4 classes
- **What to do:** In the training cell, set `num_classes` (and in the config override) to your actual number of classes (e.g. `num_classes=5` for 5 classes).

---

## Ready to run

Run the cells in order. The notebook will:

1. Mount Google Drive and check GPU/Python
2. Clone ModelAssistant and install dependencies
3. Download pretrained weights and copy your dataset from Drive
4. Train Swift-YOLO (default: 3 epochs for a quick test; use 100+ for real training)
5. Zip the output folder so you can download checkpoints and exports

## 1. Check GPU and Python version

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# Check if GPU is available
!nvidia-smi

# Check Python version
!python --version

## 2. Prerequisites: clone ModelAssistant and install SSCMA
Clone the [ModelAssistant](https://github.com/Seeed-Studio/ModelAssistant) repo and run the Colab setup script. You can ignore any non-fatal errors from the setup script.

In [ ]:
!pip install ethos-u-vela
!git clone https://github.com/Seeed-Studio/ModelAssistant.git -b 2.0.0  #clone the repo
%cd ModelAssistant
!. ./scripts/setup_colab.sh


## 3. Download pretrained weights

In [ ]:
%mkdir -p Swift-YOLO_192
!wget -c https://files.seeedstudio.com/sscma/model_zoo/detection/animal/animal_detection.pth -O Swift-YOLO_192/pretrain.pth

## 4. Copy dataset from Google Drive

Copies `dataset.zip` from `/content/drive/MyDrive/yolo/` into Colab and unzips it under `Swift-YOLO_192/`. Ensure you created the `yolo` folder and put `dataset.zip` there (see Setup Step 2). Overwrites if files already excist.


In [ ]:
!cp '/content/drive/MyDrive/dataset.zip' '/content/dataset.zip'
!unzip -o '/content/dataset.zip' -d '/content/ModelAssistant/Swift-YOLO_192/'

### Optional: use your own pretrained checkpoint
If you have a trained Swift-YOLO checkpoint in Drive (e.g. `best_coco_bbox_mAP_epoch_100.pth`), copy it to `Swift-YOLO_192/pretrain.pth` to fine-tune from it. Put the file in `/content/drive/MyDrive/` and adjust the filename below if needed.

In [ ]:
!cp '/content/drive/MyDrive/best_coco_bbox_mAP_epoch_100.pth' 'Swift-YOLO_192/pretrain.pth'

## 5. Train Swift-YOLO with SSCMA

Training uses the config `swift_yolo_tiny_1xb16_300e_coco.py`. Key parameters (overridden with `--cfg-options`):

| Parameter | Description |
|-----------|-------------|
| `data_root` | Path to dataset (train/valid with COCO JSONs) |
| `epochs` | Number of training epochs (**3 here = quick test**; use 100+ for real training) |
| `batch` | Batch size |
| `height`, `width` | Input size (192×192 for Swift-YOLO) |
| `load_from` | Pretrained checkpoint path |
| `num_classes` | Number of classes in your dataset |

Change these in the cell below to match your dataset and goal.

In [ ]:
!sscma.train configs/swift_yolo/swift_yolo_tiny_1xb16_300e_coco.py \
--cfg-options  \
    work_dir=Swift-YOLO_192 \
    num_classes=4 \
    epochs=3  \
    height=192 \
    width=192 \
    batch=256 \
    lr=0.16 \
    data_root=Swift-YOLO_192/dataset/ \
    load_from=Swift-YOLO_192/pretrain.pth

## 6. Zip and download the Swift-YOLO_192 folder

Packs checkpoints and exported files from `Swift-YOLO_192/` into a zip and triggers download. You can then export to `int8_vela.tflite` for Grove Vision AI V2 (see Deploy below) or use the `.pth` checkpoint for further training.

In [ ]:
import os
import zipfile
from google.colab import files

# Define the paths
folder_path = '/content/ModelAssistant/Swift-YOLO_192'
zip_filename = 'Swift-YOLO_Models.zip'

# Create the zip file
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    # List all items in the directory
    for item in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item)

        # Only add to zip if it's a file (skips all subdirectories)
        if os.path.isfile(item_path):
            zipf.write(item_path, arcname=item)
            print(f"Added to zip: {item}")

# Automatically trigger the download to your computer
files.download(zip_filename)